In [1]:
import os

import numpy as np
import pandas as pd

from typing import List

# import molnet loaders from deepchem
from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21
from rdkit import Chem
import sklearn

x:\Anaconda3\envs\pytorch3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
x:\Anaconda3\envs\pytorch3\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [4]:
#1. 数据输入和特征化
###设置
training_data_path = os.path.abspath(os.path.join(os.getcwd(), ".."))+'/training_data/'
dataset = 'TCM_100t'
tar_id = 'CHEMBL205.csv'


smiles = pd.read_csv(training_data_path + '/' +dataset+'/'+tar_id, header=0,index_col=False)['c_smiles'].tolist()
labels = pd.read_csv(training_data_path + '/' +dataset+'/'+tar_id, header=0,index_col=False)['active_label'].tolist()
df = pd.DataFrame(list(zip(smiles, labels)), columns=["smiles", "labels"])

In [6]:
# 在当前文件夹下下载chemberta预训练模型
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

fatal: destination path 'bert-loves-chemistry' already exists and is not an empty directory.


In [7]:
# 建子文件夹用于储存模型（仅构建目录，不一定要存模型）
%cd "X:\jupyter workspace\NP\DL\bert-loves-chemistry"
!mkdir BPE_PubChem_10M_TCM_run

X:\jupyter workspace\NP\DL\bert-loves-chemistry


��Ŀ¼���ļ� BPE_PubChem_10M_TCM_run �Ѿ����ڡ�


In [8]:
# 设立fine_tune模型
# 模型将自动在GPU上运行
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('roberta', 
                            'seyonec/PubChem10M_SMILES_BPE_396_250', 
                            args={'evaluate_each_epoch': True, 
                                  'evaluate_during_training_verbose': True, 
                                  'no_save': True, #是否不保存，若设置为false则保存，一个epoch有接近一个G
                                  'num_train_epochs': 10, 
                                  'auto_weights': True}) # You can set class weights by using the optional weight argument


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#数据拆分
#后面训练时需要使用dataframe作为输入，一般应该是用train_test_split方法
#由于要确保每个分层都有正负值，因此仅能使用stratifiedKFold, 而此方法必须获得X和y才能进行拆分，因此拆分后再将其拼接成为dataframe
from sklearn.model_selection import StratifiedKFold
outer_skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
X, y = df.smiles, df.labels

#交叉验证
rocauc_score = []
for train_index, val_index in outer_skf.split(X, y):
    X_train, X_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]
    train_df = pd.concat([X_train, y_train], axis=1, keys=['smiles', 'labels'])
    val_df = pd.concat([X_test, y_test], axis=1, keys=['smiles', 'labels'])


    # Train the model
    model.train_model(train_df, eval_df=val_df, 
                    output_dir=os.getcwd() + '/BPE_PubChem_10M_TCM_run',
                    args={'overwrite_output_dir': True})
    
    # 获得auroc分数
    result, model_outputs, wrong_predictions = model.eval_model(val_df, 
                                                                acc=sklearn.metrics.average_precision_score)
    

    rocauc_score.append(result['auroc'])
    print(result['auroc'])

average_rocauc_score = np.mean(rocauc_score)

x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Epoch 10 of 10: 100%|██████████| 10/10 [00:08<00:00,  1.16it/s]
x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|██████████| 6/6 [00:00<00:00, 29.20it/s]
x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0.8907563025210083


Epoch 10 of 10: 100%|██████████| 10/10 [00:07<00:00,  1.25it/s]
x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|██████████| 6/6 [00:00<00:00, 22.70it/s]
x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


1.0


Epoch 10 of 10: 100%|██████████| 10/10 [00:08<00:00,  1.24it/s]
x:\Anaconda3\envs\pytorch3\lib\site-packages\simpletransformers\classification\classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|██████████| 6/6 [00:00<00:00, 28.89it/s]

1.0


In [28]:
#数据输出
data={'targets':tar_id, 'best_params':"", 'rocauc_score':average_rocauc_score}
roc_data = pd.DataFrame(data, index=[0]) 
roc_data.to_csv(os.path.abspath(os.path.join(os.getcwd(), ".."))+'\\'+tar_id, index=False)